In [1]:
import pandas as pd
import features_processing_utils as fpu
import hopsworks

c:\Users\celin_e3kjw6h\miniconda3\envs\fake_news\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Log into Hopsworks

In [2]:
project = hopsworks.login()
# util.purge_project(proj)

2025-01-06 20:55:34,784 INFO: Initializing external client
2025-01-06 20:55:34,785 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 20:55:36,771 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1149079


In [3]:
politifact_fake = pd.read_csv("FakeNewsNet/politifact_fake.csv")
politifact_fake.head()

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...


In [4]:
politifact_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         432 non-null    object
 1   news_url   428 non-null    object
 2   title      432 non-null    object
 3   tweet_ids  392 non-null    object
dtypes: object(4)
memory usage: 13.6+ KB


In [5]:
politifact_real = pd.read_csv("FakeNewsNet/politifact_real.csv")
politifact_real.head()

,id,news_url,title,tweet_ids
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...
2,politifact333,https://web.archive.org/web/20080204072132/htt...,"Romney makes pitch, hoping to close deal : Ele...",NaN
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN
4,politifact779,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008",89804710374154240\t91270460595109888\t96039619...


In [6]:
politifact_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         624 non-null    object
 1   news_url   567 non-null    object
 2   title      624 non-null    object
 3   tweet_ids  409 non-null    object
dtypes: object(4)
memory usage: 19.6+ KB


In [7]:
politifact_fake_posts = pd.read_csv("data/politifact_fake_posts.csv")

In [8]:
# politifact_fake_posts = fpu.create_dataset(politifact_fake)
# politifact_fake_posts.to_csv("data/politifact_fake_posts.csv", index=False)

In [9]:
# politifact_fake_features = fpu.get_features(politifact_fake_posts, 0)
# politifact_fake_features.head()

In [10]:
# politifact_fake_features.info()

In [11]:
# politifact_real_posts = pd.read_csv("data/politifact_real_posts.csv")

In [12]:
# politifact_real_posts = fpu.create_dataset(politifact_real)
# politifact_real_posts.to_csv("data/politifact_real_posts.csv", index=False)

In [13]:
# politifact_real_features = fpu.get_features(politifact_real_posts, 1)
# politifact_real_features.head()

In [14]:
# politifact_real_features.info()

In [15]:
# politifact_features = pd.concat([politifact_fake_features, politifact_real_features])
# politifact_features.info()

In [16]:
# politifact_features.to_csv("data/politifact_features.csv", index=False)

In [17]:
politifact_features = pd.read_csv("data/politifact_features.csv")

In [18]:
gossipcop_fake = pd.read_csv("FakeNewsNet/gossipcop_fake.csv")

In [19]:
gossipcop_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5323 entries, 0 to 5322
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         5323 non-null   object
 1   news_url   5067 non-null   object
 2   title      5323 non-null   object
 3   tweet_ids  5135 non-null   object
dtypes: object(4)
memory usage: 166.5+ KB


In [20]:
# gossipcop_fake_posts = fpu.create_dataset(gossipcop_fake[:500])
# gossipcop_fake_posts.to_csv("data/gossipcop_fake_posts_1.csv", index=False)
# gossipcop_fake_features = fpu.get_features(gossipcop_fake_posts, 0)
# gossipcop_fake_features.to_csv("data/gossipcop_fake_features_1.csv", index=False)
# gossipcop_fake_features.head()

In [21]:
# gossipcop_fake_posts = fpu.create_dataset(gossipcop_fake[500:1000])
# gossipcop_fake_posts.to_csv("data/gossipcop_fake_posts_1_bis.csv", index=False)
# gossipcop_fake_features = fpu.get_features(gossipcop_fake_posts, 0)
# gossipcop_fake_features.to_csv("data/gossipcop_fake_features_1_bis.csv", index=False)
# gossipcop_fake_features.head()

In [22]:
# gossipcop_fake_features.info()

In [23]:
# gossipcop_fake_posts = fpu.create_dataset(gossipcop_fake[1000:2000])
# gossipcop_fake_posts.to_csv("data/gossipcop_fake_posts_2.csv", index=False)
# gossipcop_fake_features = fpu.get_features(gossipcop_fake_posts, 0)
# gossipcop_fake_features.to_csv("data/gossipcop_fake_features_2.csv", index=False)
# gossipcop_fake_features.info()

In [24]:
# fpu.complete_processing(gossipcop_fake, 0, "gossipcop_fake_posts_3.csv", "gossipcop_fake_features_3.csv", 2000, 3000)

In [25]:
gossipcop_real = pd.read_csv("FakeNewsNet/gossipcop_real.csv")

In [26]:
gossipcop_real_features_1 = fpu.complete_processing(gossipcop_real, 1, "gossipcop_real_posts_1.csv", "gossipcop_real_features_1.csv", end=1000)
gossipcop_real_features_1.info()

 11%|█         | 106/1000 [05:13<44:04,  2.96s/it] 


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [27]:
#gossicop_fake_features = fpu.complete_processing(gossipcop_fake, 0, "gossipcop_fake_posts_4.csv", "gossipcop_fake_features_4.csv", 3000, 4000)
# gossicop_fake_features.info()

In [28]:
gossipcop_real_features_2 = fpu.complete_processing(gossipcop_real, 1, "gossipcop_real_posts_2.csv", "gossipcop_real_features_2.csv", 1000, 2000)
gossipcop_real_features_2.info()

In [29]:
# fpu.complete_processing(gossipcop_fake, 0, "gossipcop_fake_posts_5.csv", "gossipcop_fake_features_5.csv", 4000, 5000)

In [30]:
# fpu.complete_processing(gossipcop_real, 1, "gossipcop_real_posts_3.csv", "gossipcop_real_features_3.csv", 2000, 3000)

In [31]:
# fpu.complete_processing(gossipcop_real, 1, "gossipcop_real_posts_4.csv", "gossipcop_real_features_4.csv", 3000, 4000)

In [32]:
# gossicop_real_features = fpu.complete_processing(gossipcop_real, 1, "gossipcop_real_posts_5.csv", "gossipcop_real_features_5.csv", 4000)
# gossicop_real_features.info()

In [33]:
#gossicop_features = pd.concat([gossicop_real_features, gossicop_fake_features])

In [34]:
# source = gossipcop_real
# label = 1, 
# posts_name = "gossipcop_real_posts_1.csv"
# feature_name = "gossipcop_real_features_1.csv"
# start = 0
# end=20

In [35]:
# posts = fpu.create_dataset(source[start:end])
# posts.to_csv("data/" + posts_name, index=False)

In [36]:
# posts.info()

In [37]:
# features = fpu.get_features(posts, label)
# features.to_csv("data/" + feature_name, index=False)

In [38]:
# features.info()

In [3]:
gossicop_fake_features_1 = pd.read_csv("data/gossipcop_fake_features_1.csv")
gossicop_fake_features_1_bis = pd.read_csv("data/gossipcop_fake_features_1_bis.csv")
gossicop_fake_features_2 = pd.read_csv("data/gossipcop_fake_features_2.csv")

In [ ]:
gossicop_features = pd.concat([gossicop_fake_features_1, gossicop_fake_features_1_bis, gossipcop_real_features_1, gossipcop_real_features_2, gossicop_fake_features_2])

In [5]:
historical_features = pd.concat([politifact_features, gossicop_features])

In [43]:
historical_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 254 entries, 0 to 46
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   news_id                    254 non-null    object 
 1   average followers          254 non-null    float64
 2   average follows            254 non-null    float64
 3   repost total               254 non-null    int64  
 4   post total                 254 non-null    int64  
 5   repost percentage          254 non-null    float64
 6   average repost             254 non-null    float64
 7   average favorite           254 non-null    float64
 8   label                      254 non-null    int64  
 9   news lifetime              254 non-null    int64  
 10  nb users 10 hours          254 non-null    int64  
 11  average time difference    254 non-null    float64
 12  retweet percentage 1 hour  254 non-null    float64
dtypes: float64(7), int64(5), object(1)
memory usage: 27.8+ K

In [44]:
historical_features.groupby("label").size()

label
0    122
1    132
dtype: int64

In [45]:
historical_features.columns

Index(['news_id', 'average followers', 'average follows', 'repost total',
       'post total', 'repost percentage', 'average repost', 'average favorite',
       'label', 'news lifetime', 'nb users 10 hours',
       'average time difference', 'retweet percentage 1 hour'],
      dtype='object')

In [46]:
# change names of columns to add underscore
historical_features.columns = historical_features.columns.str.replace(' ', '_')

In [47]:
historical_features.columns

Index(['news_id', 'average_followers', 'average_follows', 'repost_total',
       'post_total', 'repost_percentage', 'average_repost', 'average_favorite',
       'label', 'news_lifetime', 'nb_users_10_hours',
       'average_time_difference', 'retweet_percentage_1_hour'],
      dtype='object')

In [48]:
historical_features.to_csv("data/historical_features.csv", index=False)

Save all the data into a Feature Store via Hopsworks

In [49]:
fs = project.get_feature_store() 

Insert dataframe into Feature Group, ```expectation_suite``` should contain data validation rules => we need to come up withd data validation rules

Do we actually need an event time ?

In [50]:
news_propagation_fg = fs.get_or_create_feature_group(
    name='news_propagation',
    description='Propagation characteristics of news in Bsky',
    version=1,
    primary_key=['news_id'],
    online_enabled=True,
)

In [ ]:
news_propagation_fg.insert(historical_features, write_options={"wait_for_job": True})

Uploading Dataframe: 100.00% |██████████| Rows 254/254 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: news_propagation_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1149079/jobs/named/news_propagation_1_offline_fg_materialization/executions
2025-01-06 13:01:20,515 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-06 13:01:23,714 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-06 13:02:50,069 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-01-06 13:02:50,238 INFO: Waiting for log aggregation to finish.
2025-01-06 13:03:02,214 INFO: Execution finished successfully.


(Job('news_propagation_1_offline_fg_materialization', 'SPARK'), None)

Add a description for each feature in the feature group.

In [52]:
news_propagation_fg.update_feature_description("news_id", "The id of the news")
news_propagation_fg.update_feature_description("average_followers", "The average number of followers of the users who posted or reposted the news")
news_propagation_fg.update_feature_description("average_follows", "The average number of follows of the users who posted or reposted the news")
news_propagation_fg.update_feature_description("repost_total", "The total number of reposts of the news")
news_propagation_fg.update_feature_description("post_total", "The total number of posts of the news")
news_propagation_fg.update_feature_description("repost_percentage", "The percentage of reposts of the news out of the total number of posts and reposts")
news_propagation_fg.update_feature_description("average_repost", "The average number of reposts of the news")
news_propagation_fg.update_feature_description("average_favorite", "The average number of likes of the news")
news_propagation_fg.update_feature_description("label", "The label of the news : if it is fake or real")
news_propagation_fg.update_feature_description("news_lifetime", "The lifetime of the news, which is the time difference between the first and the last post (or repost) of the news")
news_propagation_fg.update_feature_description("nb_users_10_hours", "The number of users who posted (or reposted) the news in the first 10 hours")
news_propagation_fg.update_feature_description("average_time_difference", "The average time difference between a post and its reposts of the news")
news_propagation_fg.update_feature_description("retweet_percentage_1_hour", "The percentage of retweets of the news in the first hour")
